# 1. Economic Calendar
  **BeautifulSoup**

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
from datetime import datetime
import json

In [2]:
import pandas as pd

def news_verification():
    r = Request('https://investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
    response = urlopen(r).read()
    soup = bs(response, "html.parser")
    table = soup.find_all(class_ = "js-event-item")

    result = []
    base = {}
    
    for bl in table:
        time = bl.find(class_ ="first left time js-time").text
        # evento = bl.find(class_ ="left event").text
        currency = bl.find(class_ ="left flagCur noWrap").text.split(' ')
        intensity = bl.find_all(class_="left textNum sentiment noWrap")
        id_hour = currency[1] + '_' + time
         
        if not id_hour in base:
            base.update({id_hour : {'currency' : currency[1], 'time' : time,'intensity' : { "1": 0,"2": 0,"3": 0} } })
        
        intencity = base[id_hour]['intensity']
            
        for intence in intensity:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")
            
            if len(_true) == 1:
                intencity['1'] += 1
                
            elif len(_true) == 2:
                intencity['2'] += 1
                
            elif len(_true) == 3:
                intencity['3'] += 1
            
        base[id_hour].update({'intensity' : intencity})

    for b in base:
        result.append(base[b])

    return result

news = news_verification()

#print(json.dumps(news, indent=2))

df = pd.DataFrame(news)
df.head(10)

,currency,time,intensity
0,RUB,01:00,"{'1': 0, '2': 1, '3': 0}"
1,ZAR,01:00,"{'1': 2, '2': 0, '3': 0}"
2,GBP,02:00,"{'1': 0, '2': 2, '3': 0}"
3,CHF,03:00,"{'1': 0, '2': 1, '3': 0}"
4,EUR,03:00,"{'1': 2, '2': 2, '3': 0}"
5,HKD,03:30,"{'1': 1, '2': 0, '3': 0}"
6,EUR,04:00,"{'1': 1, '2': 0, '3': 0}"
7,GBP,05:00,"{'1': 1, '2': 0, '3': 0}"
8,INR,05:00,"{'1': 1, '2': 0, '3': 0}"
9,INR,06:30,"{'1': 2, '2': 0, '3': 0}"


In [ ]:
import pandas as pd

#HEADERS = headers.HEADERS
url = "https://www.investing.com/currencies/streaming-forex-rates-majors"


def getstringResult(len_list, list_tag_td, list_tag_th): # get string result
  symbol = ""
  stringResult=""
  for i in range(len_list):
    stringResult += list_tag_th[i].text + list_tag_td[i].text + symbol
  return stringResult

session = requests.Session()
request = session.get(url, headers=HEADERS)
#print(request.status_code)


soup = bs(request.content, 'lxml')
tag_section = soup.find("section", {"id":"leftColumn"})
list_tables = tag_section.find_all("table", {"class":"genTbl"})
name_table = tag_section.find("h1")

if len(list_tables)>0:
  for table in list_tables:
    try:
      list_tag_th = table.find("thead").find_all("th")
      list_tag_tr = table.find("tbody").find_all("tr")
    except:
      print("No connection!")

for tr in list_tag_tr:
  list_tag_td = tr.find_all("td")
  if len(list_tag_th) == len(list_tag_td):
    result = getstringResult(len(list_tag_td), list_tag_td, list_tag_th)

# Create a dataframe
df = pd.DataFrame(columns=["Pair", "Bid", "Ask", "High", "Low", "Change", "%Change", "Time"])
df.head(10)